In [3]:
import requests

resp = requests.get('https://api.kraken.com/0/public/Ticker?pair=XBTUSD')

print(resp.json())

{'error': [], 'result': {'XXBTZUSD': {'a': ['17466.10000', '1', '1.000'], 'b': ['17466.00000', '1', '1.000'], 'c': ['17466.00000', '0.00041937'], 'v': ['1710.73851382', '3064.82227922'], 'p': ['17306.06424', '17201.87541'], 't': [12850, 25766], 'l': ['17090.00000', '16938.40000'], 'h': ['17520.70000', '17520.70000'], 'o': '17209.50000'}}}


Ticker with network combination
https://support.kraken.com/hc/en-us/articles/201893658 


In [7]:
import requests
reqd = requests.get("https://api.kraken.com/0/public/AssetPairs")
reqd.json()

{'error': [],
 'result': {'1INCHEUR': {'altname': '1INCHEUR',
   'wsname': '1INCH/EUR',
   'aclass_base': 'currency',
   'base': '1INCH',
   'aclass_quote': 'currency',
   'quote': 'ZEUR',
   'lot': 'unit',
   'cost_decimals': 5,
   'pair_decimals': 3,
   'lot_decimals': 8,
   'lot_multiplier': 1,
   'leverage_buy': [],
   'leverage_sell': [],
   'fees': [[0, 0.26],
    [50000, 0.24],
    [100000, 0.22],
    [250000, 0.2],
    [500000, 0.18],
    [1000000, 0.16],
    [2500000, 0.14],
    [5000000, 0.12],
    [10000000, 0.1]],
   'fees_maker': [[0, 0.16],
    [50000, 0.14],
    [100000, 0.12],
    [250000, 0.1],
    [500000, 0.08],
    [1000000, 0.06],
    [2500000, 0.04],
    [5000000, 0.02],
    [10000000, 0.0]],
   'fee_volume_currency': 'ZUSD',
   'margin_call': 80,
   'margin_stop': 40,
   'ordermin': '10',
   'costmin': '0.45',
   'tick_size': '0.001',
   'status': 'online'},
  '1INCHUSD': {'altname': '1INCHUSD',
   'wsname': '1INCH/USD',
   'aclass_base': 'currency',
   'base': '

In [8]:
import requests
payLoad = {
        'pair':     '1INCHEUR',
        'interval': 1,
        'since' :   'last'
    }
resp = requests.get('https://api.kraken.com/0/public/OHLC',payLoad)

data = resp.json()
OHLC = len(data['result']['1INCHEUR'])
OHLC
# data

720

In [10]:
# get kraken server time
import requests

resp = requests.get('https://api.kraken.com/0/public/Time')

print(resp.json())

{'error': [], 'result': {'unixtime': 1671106255, 'rfc1123': 'Thu, 15 Dec 22 12:10:55 +0000'}}


In [6]:
# from countdown import countdown
import pandas as pd
import datetime
import os

def get_ohlc (pair, interval=1, since='last'):
    endpoint = 'https://api.kraken.com/0/public/OHLC'
    payLoad = {
        'pair':     pair,
        'interval': interval,
        'since' :   since
    }
    response = requests.get(endpoint, payLoad)
    data = response.json()
    OHLC = data['result'][pair]
    data = pd.DataFrame.from_records(OHLC, columns=['Time', 'Open', 'High', 'Low', 'Close', 'vwap', 'volume', 'count'])
    data['Time'] = pd.to_datetime(data['Time'], unit='s')
    data.set_index('Time',inplace=True)
    data = data.drop(['vwap', 'volume', 'count'], axis=1)
    data['Open']  = data.Open.astype(float)
    data['High']  = data.High.astype(float)
    data['Low']   = data.Low.astype(float)
    data['Close'] = data.Close.astype(float)
    return data

def load_data(pair, path):
    data = pd.read_json(path + pair + '.json' , orient='split')
    tmp = data.tail(1).index                    
    tmp = tmp.strftime('%Y-%m-%d %H:%M:%S')     
    dt = str_to_datetime(tmp[0])                
    ts = dt.timestamp()                         
    return data, ts

def str_to_datetime(datestr):
    Y = int(datestr[0:4])
    M = int(datestr[5:7])
    D = int(datestr[8:10])
    H = int(datestr[11:13])
    m = int(datestr[14:16])

    return datetime.datetime(Y, M, D, H, m, 0, tzinfo=tz.gettz("Etc/GMT"))

In [7]:
from countdown import countdown
import pandas as pd
import datetime
import os, requests
import uuid
import dateutil.parser, dateutil.tz as tz

path = os.getcwd() + '\\historical_data\\'
pair = '1INCHEUR'

while True:
    if os.path.exists(path + pair + '.json') == False:
        data = get_ohlc(pair, 1)                            # 1 minute timeframe
        # data.to_csv(r'C:\Users\User\Desktop\MLops project\Experiments\historical_data\data.csv')
        data.to_json(path + pair + '.json', orient='split')
    else:
        data1, ts = load_data(pair, path)
        data2 = get_ohlc(pair, 1, ts)
        data3 = pd.concat([data1, data2])
        data3.drop(data3.tail(1).index,inplace=True) # delete last record because it's not ended
        data3.to_json(path + pair + '.json', orient='split')
        savePath = "C:\\Users\\User\\Desktop\\MLops project\\Experiments\\historical_data\\"  + str(uuid.uuid4())  + ".csv"
        data3.to_csv(savePath)

    countdown(mins=12,secs=0) # update every hour 12 hours worth of data with every minute

KeyboardInterrupt: 

In [8]:
import json
import pandas as pd

file = 'C:/Users/User/Desktop/MLops project/Experiments/historical_data/1INCHEUR.json'
with open(file) as train_file:
    dict_train = json.load(train_file)

# converting json dataset from dictionary to dataframe
train = pd.DataFrame.from_dict(dict_train)
train.reset_index(level=0, inplace=True)

ValueError: All arrays must be of the same length

In [15]:
df = pd.read_json(r"C:\Users\User\Desktop\MLops project\Experiments\historical_data\1INCHEUR.json",orient='split')
display(df.head())
display(df.tail())

,Open,High,Low,Close
2022-12-15 00:07:00,0.401,0.401,0.401,0.401
2022-12-15 00:08:00,0.401,0.401,0.401,0.401
2022-12-15 00:09:00,0.401,0.401,0.401,0.401
2022-12-15 00:10:00,0.401,0.401,0.401,0.401
2022-12-15 00:11:00,0.401,0.401,0.401,0.401


,Open,High,Low,Close
2022-12-15 12:02:00,0.411,0.411,0.411,0.411
2022-12-15 12:03:00,0.411,0.411,0.411,0.411
2022-12-15 12:04:00,0.411,0.411,0.411,0.411
2022-12-15 12:05:00,0.411,0.411,0.411,0.411
2022-12-15 12:06:00,0.411,0.411,0.411,0.411


In [5]:
import requests

resp = requests.get('https://api.kraken.com/0/public/Trades?pair=1INCHEUR')

dfJSon = resp.json()


ValueError: Invalid file path or buffer object type: <class 'dict'>